In [1]:
import numpy as np 
import pandas as pd 

In [2]:
train_data_path = './data/d_training_set.csv' 
test_data_path = './data/d_test_set.csv' 

train_data = pd.read_csv(train_data_path) 
test_data = pd.read_csv(test_data_path)

In [3]:
def process_data(dframe, test=None):
    df = dframe
    print('Data Processing Started')
    if not test: 
        df= df.drop('row_id', axis=1)
        #X, y = df.loc[:, df.columns != 'repaid_loan'], df['repaid_loan'] 
        cat_features = (df.dtypes == 'object') 
        cat_features = list(cat_features[cat_features].index) 
        num_features = (df.dtypes != 'object') 
        num_features = list(num_features[num_features].index) 

        df_cat = df[cat_features] 
        df_num  = df[num_features] 

        df_cat.fillna('Unknown', axis=1, inplace=True)
        # Additional numerical features in the dataset 
        num_features_2 = ['term', 'int_rate', 'emp_length', 'issue_d', 'revol_util'] 

        # Replacing the percentage symbol with an empty string 
        df_cat['int_rate'] = df_cat['int_rate'].replace({'%': ''}, regex=True) 
        # convert the string to a float 
        df_cat['int_rate'] = df_cat['int_rate'].astype(float) 
        # Replacing string symbols with integers 
        df_cat['term'] = df_cat['term'].replace({' 36 months': 36, ' 60 months': 60}, regex=True)
        df_cat['emp_length'] = df_cat['emp_length'].replace({'years': '', '10+': 10, 'year': '', '< 1 year': 1}, regex=True)
        # drop the 'issue_d' column as we deem it unnecessary for the model 
        df_cat.drop('issue_d', axis=1, inplace=True)
        df_cat['revol_util'] = df_cat['revol_util'].replace({'%': ''}, regex=True) 
        df_cat['revol_util'] = df_cat['revol_util'].replace({'Unknown': 0}, regex=True)
        df_cat['revol_util'] = df_cat['revol_util'].astype(float) 
        df_cat['term'] = df_cat['term'].astype(float)
        num_features_2.remove('issue_d')

        df_num_2 = df_cat[num_features_2] 
        df_num = pd.concat([df_num, df_num_2], axis=1)
        # Filling missing variables with the mean of each column 
        df_num.fillna(df_num.mean(), inplace=True) 
        # drop the new numeric features from the categorical dataframe
        df_cat = df_cat.drop(num_features_2, axis=1)

        df_num['emp_length'] = df_num['emp_length'].replace({'< 1': 1}, regex=True) 
        df_num = df_num.replace({'Unknown': 0}, regex=True)

        df_combined = pd.concat([df_num, df_cat], axis=1)
        # categorical features (or attributes) 
        cat_attribs = list(df_cat.columns) 
        num_attribs = list(df_num.columns) 
        if 'repaid_loan' in num_attribs:
            num_attribs.remove('repaid_loan')
        print('*************Training data processed!!*************')
        return df_combined, cat_attribs, num_attribs

    elif test: 
        # if it's the test set 
        if 'row_id' in list(df.columns):
            
            test_row_id = df[['row_id']].copy() 
            df=df.drop('row_id', axis=1)
        cat_features = (df.dtypes == 'object') 
        cat_features = list(cat_features[cat_features].index) 
        num_features = (df.dtypes != 'object') 
        num_features = list(num_features[num_features].index) 

        df_cat = df[cat_features] 
        df_num  = df[num_features] 

        df_cat.fillna('Unknown', axis=1, inplace=True)
        # Additional numerical features in the dataset 
        num_features_2 = ['term', 'int_rate', 'emp_length', 'issue_d', 'revol_util'] 

        # Replacing the percentage symbol with an empty string 
        df_cat['int_rate'] = df_cat['int_rate'].replace({'%': ''}, regex=True) 
        # convert the string to a float 
        df_cat['int_rate'] = df_cat['int_rate'].astype(float) 
        # Replacing string symbols with integers 
        df_cat['term'] = df_cat['term'].replace({' 36 months': 36, ' 60 months': 60}, regex=True)
        df_cat['emp_length'] = df_cat['emp_length'].replace({'years': '', '10+': 10, 'year': '', '< 1 year': 1}, regex=True)
        # drop the 'issue_d' column as we deem it unnecessary for the model 
        df_cat.drop('issue_d', axis=1, inplace=True)
        df_cat['revol_util'] = df_cat['revol_util'].replace({'%': ''}, regex=True) 
        df_cat['revol_util'] = df_cat['revol_util'].replace({'Unknown': 0}, regex=True)
        df_cat['revol_util'] = df_cat['revol_util'].astype(float) 
        df_cat['term'] = df_cat['term'].astype(float)
        num_features_2.remove('issue_d')

        df_num_2 = df_cat[num_features_2] 
        df_num = pd.concat([df_num, df_num_2], axis=1)
        # Filling missing variables with the mean of each column 
        df_num.fillna(df_num.mean(), inplace=True) 
        # drop the new numeric features from the categorical dataframe
        df_cat = df_cat.drop(num_features_2, axis=1)

        df_num['emp_length'] = df_num['emp_length'].replace({'< 1': 1}, regex=True) 
        df_num = df_num.replace({'Unknown': 0}, regex=True)

        df_combined = pd.concat([df_num, df_cat], axis=1)
        # categorical features (or attributes) 
        cat_attribs = list(df_cat.columns) 
        num_attribs = list(df_num.columns) 
        print('*************Test data processed!!***************')
        return df_combined, cat_attribs, num_attribs, test_row_id


In [4]:
train_data.shape

(103887, 93)

In [5]:
test_data.shape

(15000, 92)

In [8]:
#concat_data = pd.concat([train_data, test_data], axis=0)

In [6]:
X_train_proc, cat_attribs, num_attribs = process_data(train_data)

Data Processing Started


/home/iron_onet/Documents/Courses/algotrade/venv/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
/tmp/ipykernel_22610/4000799705.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['int_rate'] = df_cat['int_rate'].replace({'%': ''}, regex=True)
/tmp/ipykernel_22610/4000799705.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

*************Training data processed!!*************


In [7]:
'repaid_loan' in num_attribs

False

In [8]:
X_train_num, y_train = X_train_proc[num_attribs], X_train_proc['repaid_loan']

In [9]:
X_test_proc, _, _, row_id_col = process_data(test_data, test=True)
X_test_num = X_test_proc[num_attribs]

Data Processing Started


/tmp/ipykernel_22610/4000799705.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['int_rate'] = df_cat['int_rate'].replace({'%': ''}, regex=True)
/tmp/ipykernel_22610/4000799705.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['int_rate'] = df_cat['int_rate'].astype(float)
/tmp/ipykernel_22610/4000799705.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

*************Test data processed!!***************


/tmp/ipykernel_22610/4000799705.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['revol_util'] = df_cat['revol_util'].replace({'%': ''}, regex=True)
/tmp/ipykernel_22610/4000799705.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['revol_util'] = df_cat['revol_util'].replace({'Unknown': 0}, regex=True)
/tmp/ipykernel_22610/4000799705.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [11]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.compose import ColumnTransformer

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('num' , num_pipeline, num_attribs), 

])

train_data_prepared = full_pipeline.fit_transform(X_train_num)

In [12]:
train_data_prepared.shape

(103887, 77)

In [13]:
test_data_prepared = full_pipeline.fit_transform(X_test_num)

In [14]:
test_data_prepared.shape

(15000, 77)

In [15]:
from sklearn.linear_model import LinearRegression 

lin_reg = LinearRegression() 
lin_reg.fit(train_data_prepared, y_train)

LinearRegression()

In [17]:
some_data = X_train_num.iloc[:5] 
some_labels = y_train.iloc[:5] 
some_data_prepared = full_pipeline.transform(some_data) 

print("Predictions: ", lin_reg.predict(some_data_prepared))

Predictions:  [0.94908992 0.96364431 0.80458191 0.85238397 0.76896628]


In [18]:
from sklearn.metrics import mean_squared_error 

loan_predictions = lin_reg.predict(train_data_prepared) 
lin_mse = mean_squared_error(y_train, loan_predictions) 
lin_rmse = np.sqrt(lin_mse) 
lin_rmse

0.3687883993837638

In [19]:
from sklearn.metrics import mean_absolute_error 

lin_mae = mean_absolute_error(y_train, loan_predictions) 
lin_mae

0.27378774773857867

In [20]:
from sklearn.tree import DecisionTreeRegressor 

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(train_data_prepared, y_train)

DecisionTreeRegressor(random_state=42)

In [21]:
loan_predictions = tree_reg.predict(train_data_prepared) 
tree_mse = mean_squared_error(y_train, loan_predictions) 
tree_rmse = np.sqrt(tree_mse)
tree_mse

0.0

In [22]:
from sklearn.model_selection import cross_val_score 

scores = cross_val_score(tree_reg, train_data_prepared, y_train, scoring="neg_mean_squared_error", cv=10) 
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores: ", scores) 
    print("Mean: ", scores.mean()) 
    print("STD: ", scores.std()) 
    
display_scores(tree_rmse_scores)

Scores:  [0.5296121  0.52924848 0.53449676 0.52979382 0.53278319 0.52605609
 0.52523205 0.53470255 0.52672147 0.52708686]
Mean:  0.5295733367662984
STD:  0.0032640649660648233


In [23]:
from sklearn.ensemble import RandomForestRegressor 

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42) 
forest_reg.fit(train_data_prepared, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [24]:
forest_reg_predictions = forest_reg.predict(train_data_prepared) 
forest_mse = mean_squared_error(y_train, forest_reg_predictions) 
forest_rmse = np.sqrt(forest_mse) 
forest_rmse

0.16508942100281815

In [25]:
from sklearn.model_selection import cross_val_score 

forest_scores = cross_val_score(forest_reg, train_data_prepared, y_train, 
                               scoring="neg_mean_squared_error", cv=10) 
forest_rmse_scores = np.sqrt(-forest_scores) 
display_scores(forest_rmse_scores)

Scores:  [0.38538364 0.39301706 0.39430077 0.3905664  0.39116851 0.39145385
 0.39050108 0.39465406 0.39210784 0.39309353]
Mean:  0.39162467453940353
STD:  0.0024947662940969072


In [27]:
from sklearn.model_selection import GridSearchCV 

param_grid = [
    # try 12 (3x4) combinations of hyperparameters 
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, 
    {'bootstrap' : [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

forest_reg = RandomForestRegressor(random_state=42) 
# train accross 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, 
                          scoring='neg_mean_squared_error', return_train_score=True) 
grid_search.fit(train_data_prepared, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [28]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [29]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)

In [30]:
# cross validation results 
cvres = grid_search.cv_results_ 
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]): 
    print(np.sqrt(-mean_score), params)

0.432061543336738 {'max_features': 2, 'n_estimators': 3}
0.38906241953925563 {'max_features': 2, 'n_estimators': 10}
0.3767310916463088 {'max_features': 2, 'n_estimators': 30}
0.4302146738012032 {'max_features': 4, 'n_estimators': 3}
0.3890718173181303 {'max_features': 4, 'n_estimators': 10}
0.37615580586403147 {'max_features': 4, 'n_estimators': 30}
0.4305501526654589 {'max_features': 6, 'n_estimators': 3}
0.3891503531781742 {'max_features': 6, 'n_estimators': 10}
0.37617664295804354 {'max_features': 6, 'n_estimators': 30}
0.43076492991936877 {'max_features': 8, 'n_estimators': 3}
0.38902169140716103 {'max_features': 8, 'n_estimators': 10}
0.37614317295857935 {'max_features': 8, 'n_estimators': 30}
0.4295615588557637 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.38886504889365825 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
0.4305141019121064 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
0.38835974222037983 {'bootstrap': False, 'max_features'

In [31]:
from sklearn.model_selection import RandomizedSearchCV 
from scipy.stats import randint 

param_distribs = {
    'n_estimators': randint(low=1, high=200), 
    'max_features': randint(low=1, high=8),
}

forest_reg = RandomForestRegressor(random_state=42) 
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs, 
                               n_iter=10, cv=5, scoring="neg_mean_squared_error", random_state=42) 
rnd_search.fit(train_data_prepared, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f89b3dab2b0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f89b3f06f70>},
                   random_state=42, scoring='neg_mean_squared_error')

In [32]:
cv_res = rnd_search.cv_results_ 
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]): 
    print(np.sqrt(-mean_score), params)

0.432061543336738 {'max_features': 2, 'n_estimators': 3}
0.38906241953925563 {'max_features': 2, 'n_estimators': 10}
0.3767310916463088 {'max_features': 2, 'n_estimators': 30}
0.4302146738012032 {'max_features': 4, 'n_estimators': 3}
0.3890718173181303 {'max_features': 4, 'n_estimators': 10}
0.37615580586403147 {'max_features': 4, 'n_estimators': 30}
0.4305501526654589 {'max_features': 6, 'n_estimators': 3}
0.3891503531781742 {'max_features': 6, 'n_estimators': 10}
0.37617664295804354 {'max_features': 6, 'n_estimators': 30}
0.43076492991936877 {'max_features': 8, 'n_estimators': 3}
0.38902169140716103 {'max_features': 8, 'n_estimators': 10}
0.37614317295857935 {'max_features': 8, 'n_estimators': 30}
0.4295615588557637 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
0.38886504889365825 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
0.4305141019121064 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
0.38835974222037983 {'bootstrap': False, 'max_features'

In [33]:
feature_importances = grid_search.best_estimator_.feature_importances_

In [34]:
feature_importances

array([0.01568481, 0.01588435, 0.01625727, 0.02243768, 0.02269805,
       0.025423  , 0.00383759, 0.00650887, 0.01420588, 0.00982911,
       0.0118139 , 0.00326612, 0.0201668 , 0.01689525, 0.00129296,
       0.01068757, 0.00144675, 0.00150919, 0.00028023, 0.00721264,
       0.02159454, 0.0075509 , 0.0093202 , 0.00601035, 0.00847495,
       0.01839686, 0.01736784, 0.01921478, 0.00660651, 0.01123777,
       0.02250359, 0.01958563, 0.02152768, 0.00869568, 0.00914581,
       0.01134226, 0.01570578, 0.02314643, 0.02344096, 0.02126301,
       0.00059958, 0.00052399, 0.02259178, 0.02400982, 0.01564781,
       0.01355856, 0.00836929, 0.01804774, 0.00917342, 0.01506943,
       0.01100826, 0.00475165, 0.01017774, 0.01141671, 0.01102994,
       0.01371695, 0.01494235, 0.01193035, 0.01530799, 0.01110247,
       0.01255239, 0.00141829, 0.00018546, 0.00184773, 0.00942744,
       0.01277804, 0.01130618, 0.00247469, 0.00181903, 0.02255386,
       0.01931955, 0.02256703, 0.01774655, 0.00515072, 0.05298

In [35]:
feature_importances.shape

(77,)

In [36]:
final_tree_model = grid_search.best_estimator_ 
final_test_predictions = final_tree_model.predict(X_test_num)

In [37]:
final_test_predictions

array([0.5       , 0.4       , 0.4       , ..., 0.43333333, 0.36666667,
       0.5       ])

In [39]:
from sklearn.neural_network import MLPRegressor 


mlp = MLPRegressor(solver='sgd', max_iter=100, activation='relu', 
                  random_state=42, learning_rate_init=0.01, 
                  batch_size=train_data_prepared.shape[0], momentum= 0.04)
mlp.fit(train_data_prepared, y_train)

/home/iron_onet/Documents/Courses/algotrade/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(batch_size=103887, learning_rate_init=0.01, max_iter=100,
             momentum=0.04, random_state=42, solver='sgd')

In [42]:
mlp_scores = cross_val_score(mlp, train_data_prepared, y_train, 
                               scoring="neg_mean_squared_error", cv=10) 
mlp_rmse_scores = np.sqrt(-forest_scores) 
display_scores(mlp_rmse_scores)

/home/iron_onet/Documents/Courses/algotrade/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:542: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn("Got `batch_size` less than 1 or larger than "
/home/iron_onet/Documents/Courses/algotrade/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/iron_onet/Documents/Courses/algotrade/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:542: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn("Got `batch_size` less than 1 or larger than "
/home/iron_onet/Documents/Courses/algotrade/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Sto

Scores:  [0.38538364 0.39301706 0.39430077 0.3905664  0.39116851 0.39145385
 0.39050108 0.39465406 0.39210784 0.39309353]
Mean:  0.39162467453940353
STD:  0.0024947662940969072


/home/iron_onet/Documents/Courses/algotrade/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [51]:
# Concatenate row_id and repaid_loan columns 
# save it to a csv file called loan_predictions.csv
predictions_pd = pd.DataFrame(final_test_predictions, columns=['repaid_loan'])
predictions_csv = pd.concat([row_id_col, predictions_pd], axis=1)
predictions_csv.to_csv('loan_predictions.csv', header=False, index=False)

In [54]:
# Lets try Boosting algorithms 
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score 
from sklearn.ensemble import AdaBoostRegressor 

num_trees = 30 
kfold = KFold(n_splits=10) 
model = AdaBoostRegressor(n_estimators=num_trees, random_state=42) 
results = cross_val_score(model, X_train_num, y_train, cv=kfold) 
print(results.mean())


0.06861467940611984
